In [1]:
import os
os.chdir('../')

In [2]:
from rsna_retro.imports import *
from rsna_retro.metadata import *
from rsna_retro.preprocess import *
from rsna_retro.train import *

Loading imports


In [3]:
torch.cuda.set_device(4)

## Load Model

In [4]:
name = 'baseline_any_xse_resnext_512_appian_try4_alb'

In [5]:
import pretrainedmodels

In [6]:
m = pretrainedmodels.se_resnext50_32x4d(num_classes=6, pretrained=None)

In [7]:
m.avg_pool = nn.AdaptiveAvgPool2d(1)

In [8]:
# state['model'].keys()

In [9]:
# sp2 = '~/kaggle/rsna_retro/models/model100/fold0_ep2.pt'
sp3 = '~/kaggle/rsna_retro/models/model100/fold0_ep3.pt'

saved_path = Path(sp3).expanduser()
state = torch.load(saved_path, map_location='cpu')
m.load_state_dict(state['model'])

<All keys matched successfully>

In [10]:
dls = get_album_data(48, 512, splits=Meta.splits_stg1, img_dir=path_jpg)
learn = get_learner(dls, m)

## Training

In [11]:
do_fit(learn, 4, 5e-4, freeze=True)
learn.save(f'runs/{name}-1')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.067610,0.075659,0.976926,0.954561,3:08:45


epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.068321,0.077435,0.977062,0.955172,3:08:03
1,0.069904,0.074393,0.977060,0.954587,3:08:50
2,0.060532,0.067914,0.978844,0.958915,3:09:00
3,0.053184,0.069393,0.979068,0.959666,3:09:00


In [14]:
rest, cv = Meta.splits_stg1
splits_all = [rest+cv, []]

In [15]:
learn.load(f'runs/{name}-1')
learn.dls = get_album_data(48, 512, splits=splits_all, img_dir=path_jpg)

In [16]:
do_fit(learn, 2, 1e-4, freeze=False)
learn.save(f'runs/{name}-2')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.053328,None,None,None,3:22:51
1,0.051065,None,None,None,3:23:02


/home/ashaw/anaconda3/envs/rsna_retro/lib/python3.7/site-packages/fastprogress/fastprogress.py:74: UserWarning: Your generator is empty.
  warn("Your generator is empty.")


## Submission

In [11]:
learn.load(f'runs/{name}-2')
sub_fn = f'subm/{name}'

In [12]:
learn.dls = get_test_data(Meta.df_tst, bs=128, sz=None, tst_dir='tst_jpg')

In [13]:
ob = learn.dls.one_batch()

In [15]:
preds,targs = learn.get_preds()

In [16]:
pred_csv = submission(Meta.df_tst, preds, fn=sub_fn)

In [17]:
api.competition_submit(f'{sub_fn}.csv', f'{name} appian se_resnext50 album stg2', 'rsna-intracranial-hemorrhage-detection')

100%|██████████| 26.0M/26.0M [00:03<00:00, 8.61MB/s]


Successfully submitted to RSNA Intracranial Hemorrhage Detection

In [18]:
api.competitions_submissions_list('rsna-intracranial-hemorrhage-detection')[0]

{'ref': 14455465,
 'totalBytes': 27277209,
 'date': '2020-02-12T00:48:18.387Z',
 'description': 'baseline_any_xse_resnext_512_appian_try4_alb appian se_resnext50 album stg2',
 'errorDescription': None,
 'fileName': 'baseline_any_xse_resnext_512_appian_try4_alb.csv',
 'publicScore': None,
 'privateScore': None,
 'status': 'pending',
 'submittedBy': 'Andrew Shaw',
 'submittedByRef': 'bearpelican',
 'teamName': 'Andrew Shaw',
 'type': 'standard',
 'url': 'https://www.kaggle.com/submissions/14455465/14455465.raw'}

## Save Predictions

## NEED TO DO ALBUMENTATION

In [11]:
path_appfeat512 = path/'appian_features_512_album'
path_appfeat512_tst = path/'appian_tst_features_512_album'

In [12]:
fn_save = f'runs/{name}-2'

In [25]:
dls = get_test_data(Meta.df_tst, bs=256, sz=None, tst_dir='tst_jpg')
m = pretrainedmodels.se_resnext50_32x4d(num_classes=6, pretrained=None)
learn = get_learner(dls, m, lf=DummyLoss(), metrics=[])
learn.load(fn_save)
learn.model.avg_pool = nn.AdaptiveAvgPool2d((4,4))
learn.model.last_linear = nn.Identity()
save_features(learn, path_appfeat512_tst)

In [26]:
dls = get_test_data(Meta.df_comb, bs=128, sz=None, tst_dir='nocrop_jpg', sl=slice(0,300000))
m = pretrainedmodels.se_resnext50_32x4d(num_classes=6, pretrained=None)
learn = get_learner(dls, m, lf=DummyLoss(), metrics=[])
learn.load(fn_save)
learn.model.avg_pool = nn.AdaptiveAvgPool2d((4,4))
learn.model.last_linear = nn.Identity()
save_features(learn, path_appfeat512)

In [13]:
dls = get_test_data(Meta.df_comb, bs=128, sz=None, tst_dir='nocrop_jpg', sl=slice(0,300000))
m = pretrainedmodels.se_resnext50_32x4d(num_classes=6, pretrained=None)
learn = get_learner(dls, m, lf=DummyLoss(), metrics=[])
learn.load(fn_save)
learn.model.avg_pool = nn.AdaptiveAvgPool2d((4,4))
learn.model.last_linear = nn.Identity()
save_features(learn, path_appfeat512)

In [15]:
import gc
gc.collect()

3846